In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("D:\study\ML\diabetes.csv")

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [5]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_skl = scaler.fit_transform(X)

C:\Users\User\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [7]:
X_skl.shape

(768, 8)

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X_skl,y,test_size=0.2,random_state=1)

In [9]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [10]:
model = Sequential()

model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [11]:
model.fit(x_train,y_train,batch_size=32,epochs=10,validation_data=(x_test,y_test))

Epoch 1/10


20/20 [==============================] - 1s 18ms/step - loss: 0.7032 - accuracy: 0.5505 - val_loss: 0.6821 - val_accuracy: 0.6623
Epoch 2/10
20/20 [==============================] - 0s 6ms/step - loss: 0.6511 - accuracy: 0.6629 - val_loss: 0.6365 - val_accuracy: 0.7078
Epoch 3/10
20/20 [==============================] - 0s 6ms/step - loss: 0.6115 - accuracy: 0.6906 - val_loss: 0.6016 - val_accuracy: 0.7078
Epoch 4/10
20/20 [==============================] - 0s 5ms/step - loss: 0.5806 - accuracy: 0.7117 - val_loss: 0.5748 - val_accuracy: 0.7208
Epoch 5/10
20/20 [==============================] - 0s 5ms/step - loss: 0.5565 - accuracy: 0.7231 - val_loss: 0.5531 - val_accuracy: 0.7468
Epoch 6/10
20/20 [==============================] - 0s 5ms/step - loss: 0.5365 - accuracy: 0.7345 - val_loss: 0.5381 - val_accuracy: 0.7662
Epoch 7/10
20/20 [==============================] - 0s 6ms/step - loss: 0.5216 - accuracy: 0.7508 - val_loss: 0.5260 - val_accuracy: 0.7597
Epoch 8/10
20/20 

In [15]:
# pip install keras-tuner

In [18]:
import kerastuner as kt

# Tuner for selecting best optimizer

In [21]:
# First create build_model function
# hp is hyperparameter
def build_model(hp):
    # building a model
    model = Sequential()
    model.add(Dense(32,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))
    
    optimizers = hp.Choice('optimizer',values = ['adam','sgd','rmsprop','adadelta'])
    model.compile(optimizer=optimizers,loss='binary_crossentropy',metrics=['accuracy'])
    
    return model

In [22]:
tuner = kt.RandomSearch(build_model,
                       objective ='val_accuracy',
                       max_trials=5)

In [23]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 4 Complete [00h 00m 01s]
val_accuracy: 0.5974025726318359

Best val_accuracy So Far: 0.7792207598686218
Total elapsed time: 00h 00m 07s


In [24]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [27]:
# to get best performing model
model = tuner.get_best_models(num_models=1)[0]
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [30]:
model.fit(x_train,y_train,batch_size=32,epochs=100,initial_epoch=5,validation_data=(x_test,y_test))

Epoch 6/100
20/20 [==============================] - 1s 12ms/step - loss: 0.5268 - accuracy: 0.7427 - val_loss: 0.5170 - val_accuracy: 0.7792
Epoch 7/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5112 - accuracy: 0.7492 - val_loss: 0.5037 - val_accuracy: 0.7857
Epoch 8/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4982 - accuracy: 0.7590 - val_loss: 0.4939 - val_accuracy: 0.7922
Epoch 9/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4893 - accuracy: 0.7492 - val_loss: 0.4896 - val_accuracy: 0.7857
Epoch 10/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4821 - accuracy: 0.7573 - val_loss: 0.4845 - val_accuracy: 0.7857
Epoch 11/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4760 - accuracy: 0.7687 - val_loss: 0.4815 - val_accuracy: 0.7792
Epoch 12/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4718 - accuracy: 0.7671 - val_loss: 0.4808 - val_accuracy: 0.7792
Epoch 13/

Epoch 64/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4303 - accuracy: 0.7883 - val_loss: 0.4784 - val_accuracy: 0.7857
Epoch 65/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4302 - accuracy: 0.7850 - val_loss: 0.4789 - val_accuracy: 0.7857
Epoch 66/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4302 - accuracy: 0.7932 - val_loss: 0.4801 - val_accuracy: 0.7857
Epoch 67/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4296 - accuracy: 0.7915 - val_loss: 0.4783 - val_accuracy: 0.7857
Epoch 68/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4293 - accuracy: 0.7899 - val_loss: 0.4788 - val_accuracy: 0.7857
Epoch 69/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4290 - accuracy: 0.7850 - val_loss: 0.4788 - val_accuracy: 0.7857
Epoch 70/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4287 - accuracy: 0.7915 - val_loss: 0.4784 - val_accuracy: 0.7857
Epoch 